In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import numpy as np

In [ ]:
playa_att_df = pd.read_csv('../data/playa_nogeometry_whucs.csv')

In [ ]:
# Convert HUC and author to int codes: (using huc8 instead of huc12)
playa_att_df['huc8'] = playa_att_df['huc12'].astype(str).str[0:8]
playa_att_df['huc8'] = playa_att_df['huc8'].astype('category').cat.codes

In [ ]:
huc_states = playa_att_df[['state','huc8']].groupby('huc8').first()

In [ ]:
id_metadata = pd.DataFrame({'id':range(playa_att_df['id'].max())})
id_metadata = id_metadata.set_index('id').join(
    playa_att_df[['id','state', 'hydromod','farmed','healthy']].groupby('id').first(), how='outer')
id_metadata = id_metadata.fillna('NA')

In [ ]:
writer = SummaryWriter()

In [ ]:
model_weights_dict = torch.load('./model_weights.pytorch',map_location=torch.device('cpu'))

In [ ]:
writer.add_embedding(model_weights_dict['author_embedding.weight'], tag='author',
                     metadata=['PLJV', 'RMBO', 'Texas Tech University', 'USFWS', 'University of Kansas'])
writer.add_embedding(model_weights_dict['huc_embedding.weight'], tag='huc',
                    metadata=huc_states['state'].values)
writer.add_embedding(model_weights_dict['id_embedding.weight'], tag='id',
                     metadata=id_metadata['state'].values)
writer.add_embedding(model_weights_dict['id_embedding.weight'], tag='id_hydromod',
                     metadata=id_metadata['hydromod'].values)
writer.add_embedding(model_weights_dict['id_embedding.weight'], tag='id_healthy',
                     metadata=id_metadata['healthy'].values)
writer.add_embedding(model_weights_dict['id_embedding.weight'], tag='id_farmed',
                     metadata=id_metadata['farmed'].values)
writer.close()
